In [46]:
import numpy as np 
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_absolute_error

In [31]:
# 2. Cargar y preprocesar datos
df = pd.read_csv('../tableClaudePeriod.csv')

In [32]:
#Ver dataFrame
df

,periodo,sem1,sem2,sem3,sem4,sem5,sem6,sem7,sem8,sem9
0,2012-2,0,15,18,12,18,0,0,0,0
1,2013-1,2,24,19,13,14,12,0,0,0
2,2013-2,3,19,22,17,12,12,12,10,0
3,2014-1,1,19,18,18,12,12,13,15,2
4,2014-2,4,19,18,18,12,12,14,15,2
5,2015-1,2,32,18,18,17,12,12,12,7
6,2015-2,2,14,24,17,18,14,12,11,6
7,2016-1,1,26,14,23,16,18,12,12,5
8,2016-2,2,22,13,22,18,14,19,13,4
9,2017-1,2,25,15,13,12,14,12,12,6


In [6]:
# Reemplaza los valores 0 por NaN en todo el DataFrame
#df.replace(0, np.nan, inplace=True)

In [7]:
#Imprimir de nuevo DataFrame
df

,periodo,sem1,sem2,sem3,sem4,sem5,sem6,sem7,sem8,sem9
0,2012-2,NaN,15,18,12,18,NaN,NaN,NaN,NaN
1,2013-1,2.0,24,19,13,14,12.0,NaN,NaN,NaN
2,2013-2,3.0,19,22,17,12,12.0,12.0,10.0,NaN
3,2014-1,1.0,19,18,18,12,12.0,13.0,15.0,2.0
4,2014-2,4.0,19,18,18,12,12.0,14.0,15.0,2.0
5,2015-1,2.0,32,18,18,17,12.0,12.0,12.0,7.0
6,2015-2,2.0,14,24,17,18,14.0,12.0,11.0,6.0
7,2016-1,1.0,26,14,23,16,18.0,12.0,12.0,5.0
8,2016-2,2.0,22,13,22,18,14.0,19.0,13.0,4.0
9,2017-1,2.0,25,15,13,12,14.0,12.0,12.0,6.0


In [33]:
#Preprocesamiento de datos
df = df.sort_values('periodo') 
df = df.interpolate()


In [34]:
#Preprocesamiento de Datos 2
scaler = MinMaxScaler()
X = scaler.fit_transform(df.iloc[:, 1:]) 


In [35]:
#Definicion del vector Y
y = df.iloc[:, 1:].to_numpy() 


In [36]:
# 3. Dividir datos en train y test
split = int(0.7*len(X))
X_train = X[:split]
X_test = X[split:]
y_train = y[:split] 
y_test = y[split:]


In [37]:
#Imprimir para validar la forma del arreglo(3 o 4 dimensiones)
X_train.shape 
# X_train tiene shape (15, 9) inicialmente (resultado que salio al imprimir)


(15, 9)

In [38]:
X_train = X_train.reshape((-1, X_train.shape[1], 1, 1)) 
# Ahora X_train tiene shape (15, 9, 1, 1)

In [39]:
total_elements = X_train.size
X_train = X_train.reshape(15, 9, 1)

In [40]:
# 4. Definir y entrenar modelo LSTM 
model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]))) 
model.add(Dense(9))

model.compile(loss='mae', optimizer='adam')


In [41]:
#Entrenar el modelo con los datos X_train y y_train:
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 12.7789
Epoch 2/100
1/1 [==============================] - 0s 8ms/step - loss: 12.7665
Epoch 3/100
1/1 [==============================] - 0s 9ms/step - loss: 12.7543
Epoch 4/100
1/1 [==============================] - 0s 14ms/step - loss: 12.7424
Epoch 5/100
1/1 [==============================] - 0s 16ms/step - loss: 12.7304
Epoch 6/100
1/1 [==============================] - 0s 13ms/step - loss: 12.7185
Epoch 7/100
1/1 [==============================] - 0s 38ms/step - loss: 12.7063
Epoch 8/100
1/1 [==============================] - 0s 10ms/step - loss: 12.6938
Epoch 9/100
1/1 [==============================] - 0s 11ms/step - loss: 12.6809
Epoch 10/100
1/1 [==============================] - 0s 14ms/step - loss: 12.6673
Epoch 11/100
1/1 [==============================] - 0s 13ms/step - loss: 12.6530
Epoch 12/100
1/1 [==============================] - 0s 24ms/step - loss: 12.6378
Epoch 13/100
1/1 [=======================

In [44]:
# Hacer predicciones con el modelo
y_pred = model.predict(X_test)

1/1 [==============================] - 0s 29ms/step


In [47]:
# Evaluar predicciones
mae = mean_absolute_error(y_test, y_pred) 
print("MAE:", mae)

MAE: 7.255638892688449


In [48]:

# Inverse transform para escala original
y_pred = scaler.inverse_transform(y_pred)
y_test = scaler.inverse_transform(y_test)

# Imprimir predicciones vs reales
for i in range(len(y_test)):
  print("Real:", y_test[i], "Predicción:", y_pred[i])

Real: [ 24. 410. 211. 228.  84. 264. 247. 270. 176.] Predicción: [ 11.881686 218.70874  136.12886  129.07388   87.885635 211.75868
 132.53574  152.8754    95.68523 ]
Real: [ 18. 446. 233. 240. 120. 264. 342. 306. 160.] Predicción: [ 11.881641 218.70297  136.12457  129.0696    87.883224 211.75378
 132.53232  152.87341   95.68263 ]
Real: [ 18. 554. 222. 252. 120. 396. 228. 324. 176.] Predicción: [ 11.881661 218.71474  136.13208  129.07701   87.88789  211.76501
 132.53922  152.87997   95.68817 ]
Real: [ 18. 428. 277. 228. 120. 264. 342. 252. 176.] Predicción: [ 11.881753 218.71011  136.12927  129.07355   87.88588  211.75996
 132.53679  152.87677   95.68585 ]
Real: [ 36. 482. 266. 300. 120. 396. 342. 216. 256.] Predicción: [ 11.882179 218.75111  136.15651  129.09842   87.90192  211.79732
 132.56215  152.89822   95.70502 ]
Real: [ 18. 392. 277. 228. 114. 484. 342. 198. 224.] Predicción: [ 11.882406 218.72295  136.13852  129.07968   87.89052  211.76987
 132.5463   152.88216   95.691826]
Real

Los resultados por el modelo de prediccion no son los mas precisos.

El siguiente objetivo es optimizar el modelo, lo cual involucra una serie de pasos.


### Analisis exploratorio de datos 
- La continuacion de este proceso seguira en la carpeta llamada modelv9
